# About Dataset:
    #### 1.title: the title of a news article
    #### 2.text: the text of the article; could be incomplete
    #### 3.label: a label that marks the news article is real or fake:

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nisha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# printing the stopwords in English

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
#Loading dataset to the pandas mainframe

news_dataset = pd.read_csv("news.csv",encoding = 'unicode_escape')

In [6]:
news_dataset.shape

(6335, 4)

In [7]:
news_dataset.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [8]:
news_dataset.head(10)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary¡¦s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"¡X Kaydee King (@KaydeeKing) November 9, 2016 ...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI¡¦m not an immigrant, but my grandparents...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,¡¥Britain¡¦s Schindler¡¦ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [9]:
# merging title and text

news_dataset['New']=news_dataset['title']+"        "+news_dataset['text']

In [10]:
print(news_dataset['New'])

0       You Can Smell Hillary¡¦s Fear        Daniel Gr...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry to go to Paris in gesture of sympathy   ...
3       Bernie supporters on Twitter erupt in anger ag...
4       The Battle of New York: Why This Primary Matte...
                              ...                        
6330    State Department says it can't find emails fro...
6331    The ¡¥P¡¦ in PBS Should Stand for ¡¥Plutocrati...
6332    Anti-Trump Protesters Are Tools of the Oligarc...
6333    In Ethiopia, Obama seeks progress on peace, se...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: New, Length: 6335, dtype: object


In [11]:
print(news_dataset.head(10))

   Unnamed: 0                                              title  \
0        8476                      You Can Smell Hillary¡¦s Fear   
1       10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2        3608        Kerry to go to Paris in gesture of sympathy   
3       10142  Bernie supporters on Twitter erupt in anger ag...   
4         875   The Battle of New York: Why This Primary Matters   
5        6903                                        Tehran, USA   
6        7341  Girl Horrified At What She Watches Boyfriend D...   
7          95               ¡¥Britain¡¦s Schindler¡¦ Dies at 106   
8        4869  Fact check: Trump and Clinton at the 'commande...   
9        2909  Iran reportedly makes new push for uranium con...   

                                                text label  \
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE   
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE   
2  U.S. Secretary of State John F. Kerry said Mon...  REAL   
3  

In [12]:
news_dataset['label'] = news_dataset['label'].replace({"FAKE":"0","REAL":"1"})

In [13]:
news_dataset.head(10)

,Unnamed: 0,title,text,label,New
0,8476,You Can Smell Hillary¡¦s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0,You Can Smell Hillary¡¦s Fear Daniel Gr...
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0,Watch The Exact Moment Paul Ryan Committed Pol...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1,Kerry to go to Paris in gesture of sympathy ...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"¡X Kaydee King (@KaydeeKing) November 9, 2016 ...",0,Bernie supporters on Twitter erupt in anger ag...
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1,The Battle of New York: Why This Primary Matte...
5,6903,"Tehran, USA","\nI¡¦m not an immigrant, but my grandparents...",0,"Tehran, USA \nI¡¦m not an immigrant, ..."
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",0,Girl Horrified At What She Watches Boyfriend D...
7,95,¡¥Britain¡¦s Schindler¡¦ Dies at 106,A Czech stockbroker who saved more than 650 Je...,1,¡¥Britain¡¦s Schindler¡¦ Dies at 106 A ...
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,1,Fact check: Trump and Clinton at the 'commande...
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,1,Iran reportedly makes new push for uranium con...


In [14]:
# separating data and lable

X = news_dataset.drop('label',axis =1)
Y = news_dataset['label']

In [15]:
X.head(15)

,Unnamed: 0,title,text,New
0,8476,You Can Smell Hillary¡¦s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",You Can Smell Hillary¡¦s Fear Daniel Gr...
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,Watch The Exact Moment Paul Ryan Committed Pol...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,Kerry to go to Paris in gesture of sympathy ...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"¡X Kaydee King (@KaydeeKing) November 9, 2016 ...",Bernie supporters on Twitter erupt in anger ag...
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,The Battle of New York: Why This Primary Matte...
5,6903,"Tehran, USA","\nI¡¦m not an immigrant, but my grandparents...","Tehran, USA \nI¡¦m not an immigrant, ..."
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",Girl Horrified At What She Watches Boyfriend D...
7,95,¡¥Britain¡¦s Schindler¡¦ Dies at 106,A Czech stockbroker who saved more than 650 Je...,¡¥Britain¡¦s Schindler¡¦ Dies at 106 A ...
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,Fact check: Trump and Clinton at the 'commande...
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,Iran reportedly makes new push for uranium con...


In [16]:
Y.head(15)

0     0
1     0
2     1
3     0
4     1
5     0
6     0
7     1
8     1
9     1
10    1
11    1
12    0
13    0
14    1
Name: label, dtype: object

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [17]:
ps = PorterStemmer()

In [18]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [19]:
news_dataset['New'] = news_dataset['New'].apply(stemming)

In [20]:
news_dataset['New'].head(15)

0     smell hillari fear daniel greenfield shillman ...
1     watch exact moment paul ryan commit polit suic...
2     kerri go pari gestur sympathi u secretari stat...
3     berni support twitter erupt anger dnc tri warn...
4     battl new york primari matter primari day new ...
5     tehran usa immigr grandpar year ago arriv new ...
6     girl horrifi watch boyfriend left facetim shar...
7     britain schindler die czech stockbrok save jew...
8     fact check trump clinton command chief forum h...
9     iran reportedli make new push uranium concess ...
10    three clinton iowa glimps fire elud hillari cl...
11    donald trump shockingli weak deleg game someho...
12    strong solar storm tech risk today news oct vi...
13    way america prepar world war octob pretti fact...
14    trump take cruz lightli kill obama administr r...
Name: New, dtype: object

In [21]:
#separating the data and label
X = news_dataset['New'].values
Y = news_dataset['label'].values

In [22]:
print(X)

['smell hillari fear daniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi word unpreced thrown around often elect ought retir still unpreced nomine major polit parti go war fbi exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect fbi attack everyon obama cnn hillari peopl circul letter attack comey current media hit piec lambast target trump surpris clinton alli start run attack ad fbi fbi leadership warn entir left wing establish form lynch mob continu go hillari fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton covert struggl fbi agent obama doj peopl gone explos public new york time compar comey j edgar hoover bizarr headlin jame comey role recal hoover fbi fairli practic admit front spout nonsens boston globe publish column call comey resign outdon time editori claim sca

In [23]:
print(Y)

['0' '0' '1' ... '0' '1' '1']


In [24]:
Y.shape

(6335,)

In [25]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [26]:
print(X)

  (0, 43974)	0.030406280778131196
  (0, 43857)	0.01822745588443043
  (0, 42956)	0.03696108001838734
  (0, 42952)	0.024034584390764016
  (0, 42926)	0.033185821021669615
  (0, 42893)	0.02362340778534529
  (0, 42891)	0.008732167889638126
  (0, 42825)	0.01578993338860275
  (0, 42765)	0.016680033740024997
  (0, 42677)	0.02838518551009315
  (0, 42676)	0.016817537965368043
  (0, 42639)	0.03824335937890267
  (0, 42597)	0.04255521120615529
  (0, 42501)	0.035014962528858336
  (0, 42448)	0.03587918387508206
  (0, 42434)	0.01841318443978269
  (0, 42356)	0.013931244743587087
  (0, 42327)	0.06206026159075188
  (0, 42222)	0.011498173449732165
  (0, 42191)	0.026998841978725343
  (0, 42115)	0.028856621113808757
  (0, 42054)	0.03271006303382345
  (0, 42006)	0.0163442959791202
  (0, 41958)	0.053407815403416874
  (0, 41926)	0.07033137019999851
  :	:
  (6334, 3868)	0.02860967911964226
  (6334, 3867)	0.026460657821927903
  (6334, 3692)	0.03534586655102588
  (6334, 3685)	0.02047775375725914
  (6334, 3411)	0.

Splitting the dataset to training & test data

In [47]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, stratify=Y, random_state=2)

#### Training the Model: Logistic Regression    

In [48]:
Model = LogisticRegression() 

In [49]:
Model.fit(X_train,Y_train)

LogisticRegression()

#### Evaluation:Accuracy score

In [50]:
# accuracy score on the training data
X_train_prediction = Model.predict(X_train)
training_accuracy = accuracy_score(X_train_prediction, Y_train)

In [51]:
print('Accuracy score of the training data : ', training_accuracy)

Accuracy score of the training data :  0.95390444117028


In [52]:
# accuracy score on the test data
X_test_prediction = Model.predict(X_test)
test_accuracy = accuracy_score(X_test_prediction, Y_test)

In [53]:
print('Accuracy score of the test data : ', test_accuracy)

Accuracy score of the test data :  0.9204545454545454


#### Making a Predictive System

In [54]:
X_new = X_test[319]

prediction = Model.predict(X_new)
print(prediction)

if (prediction[0]=='0'):
  print('The news is Real')
else:
  print('The news is Fake')

['0']
The news is Real


In [55]:
print(Y_test[319])

0
